# Google SQL for SQL Server

> [Google Cloud SQL](https://cloud.google.com/sql) 是一项完全托管的关系数据库服务，具有高性能、无缝集成和出色的可扩展性。它提供 `MySQL`、`PostgreSQL` 和 `SQL Server` 数据库引擎。利用 Cloud SQL 的 Langchain 集成，扩展您的数据库应用程序以构建支持 AI 的体验。

本笔记本将介绍如何使用 `Google Cloud SQL for SQL Server` 类来存储聊天消息历史记录。

在 [GitHub](https://github.com/googleapis/langchain-google-cloud-sql-mssql-python/) 上了解更多关于该软件包的信息。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googleapis/langchain-google-cloud-sql-mssql-python/blob/main/docs/chat_message_history.ipynb)

## 开始之前

要运行此Notebook，您需要执行以下操作：

*   [创建 Google Cloud 项目](https://developers.google.com/workspace/guides/create-project)
*   [启用 Cloud SQL Admin API。](https://console.cloud.google.com/marketplace/product/google/sqladmin.googleapis.com)
*   [创建 Cloud SQL for SQL Server 实例](https://cloud.google.com/sql/docs/sqlserver/create-instance)
*   [创建 Cloud SQL 数据库](https://cloud.google.com/sql/docs/sqlserver/create-manage-databases)
*   [创建数据库用户](https://cloud.google.com/sql/docs/sqlserver/create-manage-users)（如果您选择使用 `sqlserver` 用户，则此步骤为可选）

### 🦜🔗 库安装
集成 resides in its own `langchain-google-cloud-sql-mssql` package, so we need to install it.

In [ ]:
%pip install --upgrade --quiet langchain-google-cloud-sql-mssql langchain-google-vertexai

**仅限 Colab：** 取消注释以下单元格以重新启动内核，或使用按钮重新启动内核。对于 Vertex AI Workbench，您可以使用顶部的按钮重启终端。

In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 🔐 身份验证
请登录到 Google Cloud，使用当前笔记本中登录的 IAM 用户来访问您的 Google Cloud 项目。

* 如果您正在使用 Colab 运行此笔记本，请使用下面的单元格并继续。
* 如果您正在使用 Vertex AI Workbench，请查看此处的设置说明：[here](https://github.com/GoogleCloudPlatform/generative-ai/tree/main/setup-env)。

In [1]:
from google.colab import auth

auth.authenticate_user()

### ☁ 设置您的 Google Cloud 项目

设置您的 Google Cloud 项目，以便在本笔记本中利用 Google Cloud 资源。

如果您不知道您的项目 ID，可以尝试以下方法：

* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
# @markdown Please fill in the value below with your Google Cloud project ID and then run the cell.

PROJECT_ID = "my-project-id"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### 💡 API 启用
`langchain-google-cloud-sql-mssql` 包要求您在 Google Cloud 项目中启用 [Cloud SQL Admin API](https://console.cloud.google.com/flows/enableapi?apiid=sqladmin.googleapis.com)。

In [3]:
# enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

## 基本用法

### 设置 Cloud SQL 数据库值
在 [Cloud SQL 实例页面](https://console.cloud.google.com/sql?_ga=2.223735448.2062268965.1707700487-2088871159.1707257687) 中查找您的数据库值。

In [4]:
# @title Set Your Values Here { display-mode: "form" }
REGION = "us-central1"  # @param {type: "string"}
INSTANCE = "my-mssql-instance"  # @param {type: "string"}
DATABASE = "my-database"  # @param {type: "string"}
DB_USER = "my-username"  # @param {type: "string"}
DB_PASS = "my-password"  # @param {type: "string"}
TABLE_NAME = "message_store"  # @param {type: "string"}

### MSSQLEngine 连接池

将 Cloud SQL 设置为 ChatMessageHistory 内存存储的要求和参数之一是 `MSSQLEngine` 对象。`MSSQLEngine` 配置连接池到您的 Cloud SQL 数据库，从而实现您应用程序的成功连接并遵循行业最佳实践。

要使用 `MSSQLEngine.from_instance()` 创建 `MSSQLEngine`，您只需要提供 6 个项：

1. `project_id`：Cloud SQL 实例所在的 Google Cloud 项目的项目 ID。
1. `region`：Cloud SQL 实例所在的区域。
1. `instance`：Cloud SQL 实例的名称。
1. `database`：要连接到 Cloud SQL 实例上的数据库的名称。
1. `user`：用于内置数据库身份验证和登录的数据库用户。
1. `password`：用于内置数据库身份验证和登录的数据库密码。

默认情况下，将使用[内置数据库身份验证](https://cloud.google.com/sql/docs/sqlserver/users)（使用用户名和密码访问 Cloud SQL 数据库）来进行数据库身份验证。

In [5]:
from langchain_google_cloud_sql_mssql import MSSQLEngine

engine = MSSQLEngine.from_instance(
    project_id=PROJECT_ID,
    region=REGION,
    instance=INSTANCE,
    database=DATABASE,
    user=DB_USER,
    password=DB_PASS,
)

### 初始化表

`MSSQLChatMessageHistory` 类需要一个具有特定架构的数据库表来存储聊天消息历史记录。

`MSSQLEngine` 引擎有一个名为 `init_chat_history_table()` 的辅助方法，您可以使用它来为您创建具有适当架构的表。

In [ ]:
engine.init_chat_history_table(table_name=TABLE_NAME)

### MSSQLChatMessageHistory

要初始化 `MSSQLChatMessageHistory` 类，您只需要提供三样东西：

1. `engine` - `MSSQLEngine` 引擎的实例。
1. `session_id` - 一个唯一的标识符字符串，用于指定会话 ID。
1. `table_name` : Cloud SQL 数据库中用于存储聊天消息历史的表的名称。

In [7]:
from langchain_google_cloud_sql_mssql import MSSQLChatMessageHistory

history = MSSQLChatMessageHistory(
    engine, session_id="test_session", table_name=TABLE_NAME
)
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [8]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

#### 清理
当特定会话的历史记录已过时且可以删除时，可以通过以下方式进行。

**注意：** 删除后，数据将不再存储在 Cloud SQL 中，并且将永久消失。

In [9]:
history.clear()

## 🔗 链式调用

我们可以轻松地将此消息历史类与 [LCEL Runnables](/docs/how_to/message_history) 结合使用

为此，我们将使用 [Google 的 Vertex AI 聊天模型](/docs/integrations/chat/google_vertex_ai_palm) 之一，该模型要求您在 Google Cloud 项目中 [启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
# enable Vertex AI API
!gcloud services enable aiplatform.googleapis.com

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_vertexai import ChatVertexAI

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatVertexAI(project=PROJECT_ID)

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: MSSQLChatMessageHistory(
        engine,
        session_id=session_id,
        table_name=TABLE_NAME,
    ),
    input_messages_key="question",
    history_messages_key="history",
)

In [13]:
# This is where we configure the session id
config = {"configurable": {"session_id": "test_session"}}

In [14]:
chain_with_history.invoke({"question": "Hi! I'm bob"}, config=config)

AIMessage(content=' Hello Bob, how can I help you today?')

In [15]:
chain_with_history.invoke({"question": "Whats my name"}, config=config)

AIMessage(content=' Your name is Bob.')